In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



def scaling_factor(NA,n1,n2,lam_0):

    z = np.arange(lam_0,100,0.1)
    n2overn1 = np.divide(n2,n1)
    
    if n2overn1 < 1: eps = np.multiply(-1,np.divide(np.divide(lam_0,4),(np.multiply(z,n2))))
    else: eps = np.divide(np.divide(lam_0,4),(np.multiply(z,n2)))
    eps_term = np.multiply(eps, np.subtract(2,eps))    
    
    m = np.emath.sqrt(np.subtract(np.power(n2,2),np.power(n1,2)))
    
    sf_univ = np.multiply(np.divide(n2,n1),
                          np.divide(1-eps+np.divide(m,n1)*np.emath.sqrt(eps_term),
                                    1-np.multiply(np.divide(n2,n1)**2,eps_term)))
    sf_crit = np.divide(n1-np.emath.sqrt(np.power(n1,2)-np.power(NA,2)),
                        n2-np.emath.sqrt(np.power(n2,2)-np.power(NA,2)))
    
    sf = np.zeros(len(z))
    for i in range(len(sf)):
        if n2overn1 < 1: sf[i] = np.max([np.real(sf_univ[i]),np.real(sf_crit)])
        elif n2overn1 > 1:sf[i] = np.min([np.real(sf_univ[i]),np.real(sf_crit)])
        else: sf[i]=1
    if n2overn1 > 1:   plt.ylim([n2overn1*0.9,1.1*sf_crit])
    elif n2overn1 < 1: plt.ylim([0.8*sf_crit,n2overn1*1.1])
    else: plt.ylim(0.5,1.5)
    plt.xlim([0,100])
    plt.xlabel(r'Depth ($\mu$m)')
    plt.ylabel(r'Scaling factor $\zeta$')
    plt.plot(z,sf)

print("Scaling factor calculator \n\nAdjust the sliders to set the parameters of the plot")    
  
style = {'description_width': 'initial'}
interact(scaling_factor, NA = widgets.FloatSlider(value=0.85, description='Numerical aperture ----------', 
                                        style=style, layout = Layout(width='40%'),
                                               min=0.1,
                                               max=1.4,
                                               step=0.05),
        n1 = widgets.FloatSlider(value=1, description='Immersion refractive index $n_1$', 
                                        style=style,layout = Layout(width='40%'),
                                               min=1,
                                               max=1.52,
                                               step=0.01),
        n2 = widgets.FloatSlider(value=1.33,description='Sample refractive index $n_2$ --', 
                                        style=style,layout = Layout(width='40%'),
                                               min=1,
                                               max=1.52,
                                               step=0.01),
        lam_0 = widgets.FloatSlider(value=0.52,description=r'Wavelength $\lambda_0$ ($\mu$m) ---------', 
                                        style=style,layout = Layout(width='40%'),
                                               min=0.2,
                                               max=0.9,
                                               step=0.02))



plt.show()

Scaling factor calculator 

Adjust the sliders to set the parameters of the plot


interactive(children=(FloatSlider(value=0.85, description='Numerical aperture ----------', layout=Layout(width…